In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import joblib

## Load Features


In [ ]:
X = np.load("../features/X_hog_scaled.npy")
y = np.load("../features/y.npy")

print(X.shape, y.shape)


## Global Parameters



In [ ]:
N_NEIGHBORS = 5
WEIGHTS = "distance"
METRIC = "euclidean"
THRESHOLD = 0.6

## Model Training


In [ ]:


knn = KNeighborsClassifier(
    n_neighbors=N_NEIGHBORS,
    weights=WEIGHTS,
    metric=METRIC
)

knn.fit(X, y)


## Save Model


In [ ]:

joblib.dump(knn, "../models/knn_model.pkl")


## Model Prediction with Rejection


In [ ]:
def knn_predict_with_rejection(model, X, threshold=THRESHOLD):
    distances, indices = model.kneighbors(X)
    preds = model.predict(X)

    neighbor_labels = model._y[indices]
    confidence = (neighbor_labels == preds[:, None]).mean(axis=1)

    preds[confidence < threshold] = 6
    return preds
